# 1. Imports and Loading Data
Before running, please upload the following to the "Files" tab:
- reverse_pianoroll.py
- convert.py
- Pop_Music_Midi.zip

All can be found on GitHub (github.com/conanlu/composeGAN)

In [ ]:
!pip install pretty_midi
!pip install librosa
import pretty_midi
import reverse_pianoroll
import convert
import librosa

In [ ]:
import numpy as np
import sys

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#all necessary imports: use pip install [library name] to add to environment
#this notebook was run in 2019 with tensorflow version 1.15. some functions may or may not work with tensorflow > 2.0

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from os import listdir
import glob
#import pretty_midi
#import librosa

#python script, in github repo


In [ ]:
#add songs to data
def get_songs(path):
    files = glob.glob('{}/*.mid*'.format(path))
    songs = []
    for f in files:
        try:
            data = pretty_midi.PrettyMIDI(f)
            song = data.get_piano_roll(fs=16)
            song = convert.forward(song)
            #song = np.transpose(song) - if your code matrices aren't working, try uncommenting this. the convert.py file might not be updated
            songs.append(song)
        except Exception as e:
            raise e           
    return songs

In [ ]:
#custom function to extract chroma features from song
def get_chromas(songs):
    chromas = []
    for song in songs: 
        chroma = np.zeros(shape=(np.shape(song)[0], 12))
        for i in np.arange(np.shape(song)[0]): 
            for j in np.arange(78):
                if song[i][j] > 0:
                    chroma[i][np.mod(j,12)] += 1
        #print(np.shape(chroma))
        chromas.append(chroma)
                
    return chromas

In [ ]:
!unzip Pop_Music_Midi.zip;

In [ ]:
songs = get_songs('Pop_Music_Midi')
chromas = get_chromas(songs)
print ("{} songs processed".format(len(songs)))
print ("{} songs processed".format(len(chromas)))

# 2. Setting Up GAN Model

In [ ]:
lowest_note = 0 #the index of the lowest note on the piano roll
highest_note = 78 #the index of the highest note on the piano roll
note_range = highest_note-lowest_note #the note range

num_timesteps  = 4 #This is the number of timesteps that we will create at a time
X_dim = 2*note_range*num_timesteps #This is the size of the visible layer. 
Z_dim = 12*num_timesteps
n_hidden = 50 #This is the size of the hidden layer

print(X_dim,Z_dim)

624 48


In [ ]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [ ]:
#setting up model, discriminator weights and biases
X = tf.placeholder(tf.float32, shape=[None, X_dim])


D_W1 = tf.Variable(xavier_init([X_dim+Z_dim, 512]))
D_b1 = tf.Variable(tf.zeros(shape=[512]))

D_W2 = tf.Variable(xavier_init([512, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [ ]:
#setting up model, generator weights and biases

#z is the space we're generating from
Z = tf.placeholder(tf.float32, shape=[None, Z_dim])

G_W1 = tf.Variable(xavier_init([Z_dim, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, X_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[X_dim]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [ ]:
def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob

In [ ]:
def discriminator(x,c):
    D_h1 = tf.nn.relu(tf.matmul(tf.concat([x,c],1), D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [ ]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')

        plt.imshow(sample.reshape(78, 30), cmap='Greys_r')


    return fig

In [ ]:
print (np.shape(Z))

(?, 48)


In [ ]:
G_sample = generator(Z)

In [ ]:
print(note_range)

78


In [ ]:
D_real, D_logit_real = discriminator(X,Z)

In [ ]:
D_fake, D_logit_fake = discriminator(G_sample,Z)

In [ ]:

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
D_loss = D_loss_real + D_loss_fake
G_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))
G_loss_L1 = tf.reduce_mean(tf.losses.mean_squared_error(X,G_sample))
G_loss = G_loss_fake + 100*G_loss_L1

In [ ]:
#optimizing functions
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

In [ ]:
#output midi file folder
if not os.path.exists('out/'):
    os.makedirs('out/')

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
# old comment:
#         for song in songs:
#         # The songs are stored in a time x notes format. The size of each song is timesteps_in_song x 2*note_range
#         # Here we reshape the songs so that each training example is a vector with num_timesteps x 2*note_range elements
#         song = np.array(song)
#         song = song[:np.floor(song.shape[0]/num_timesteps).astype(int)*num_timesteps]
#         song = np.reshape(song, [int(song.shape[0]/num_timesteps), song.shape[1]*num_timesteps])
#         # Train the RBM on batch_size examples at a time

# 3. Training GAN Model

In [ ]:
i = 0
num_epochs = 200000
batch_size = 10
#commented out print statements output different losses, and plotting functions plot the piano roll and chroma.
while i <= num_epochs:
    for song, chroma in zip(songs, chromas):      
        # The songs are stored in a time x notes format. The size of each song is timesteps_in_song x 2*note_range
        # Here we reshape the songs so that each training example is a vector with num_timesteps x 2*note_range elements    
        song = np.array(song)     
        #print(np.shape(song))  
        song_steps = np.floor(song.shape[0]/num_timesteps).astype(int)
        song = song[:song_steps*num_timesteps]
        song = np.reshape(song, [song_steps, song.shape[1]*num_timesteps])  
        chroma = np.array(chroma)
        #print(np.shape(chroma)
        chroma = chroma[:song_steps*num_timesteps]
        chroma = np.reshape(chroma, [song_steps, chroma.shape[1]*num_timesteps])                
        batch_size = min(batch_size,len(song))
        # Train the RBM on batch_size examples at a time
        for ind in range(0, len(song), batch_size):       
            X_mb = song[ind:ind+batch_size]
            ch = chroma[ind:ind+batch_size]
#            _, loss = sess.run([solver, vae_loss], feed_dict={X: X_mb})
            _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: ch})
            _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={X: X_mb, Z: ch})
    
            if i % 1000 == 0:
                # print('Iter: {}'.format(i))
                dloss = ('D_Loss: {:.4}'. format(D_loss_curr))
                gloss = ('G_Loss: {:.4}'. format(G_loss_curr))
                #print(dloss)
                #print(gloss)
                
#             samples = sess.run(X_samples, feed_dict={z: np.random.randn(1,z_dim)})

                samples = sess.run(G_sample, feed_dict={Z: ch}) #or here? lol i think it's here actually
#                 print(np.shape(samples), np.shape(ch))
        
                S = np.reshape(samples, (ch.shape[0]*num_timesteps, 2*note_range))
                thresh_S = S>=0.5

                thresh_S = np.transpose(thresh_S)


                C = np.reshape(ch, (ch.shape[0]*num_timesteps, 12))

                test = reverse_pianoroll.piano_roll_to_pretty_midi(convert.back(thresh_S), fs=16)
                test.write('out/{}.mid'.format(i))

            i += 1

#4. Style Transfer with New Genre Dataset

In [ ]:
#for testing, i'll be using a different dataset of MIDI files to input into the generator here.
test_song = get_songs("Classical_Music_Midi")
test_chromaz = get_chromas(test_song)

In [ ]:
#converted midi file folder
if not os.path.exists('converted/'):
    os.makedirs('converted/')

In [ ]:


i = 0

for c in test_chromaz:
    test_chroma = np.array(c)
    

    test_chroma = test_chroma[:np.floor(test_chroma.shape[0]/num_timesteps).astype(int)*num_timesteps]
    test_chroma = np.reshape(test_chroma, [int(test_chroma.shape[0]/num_timesteps), test_chroma.shape[1]*num_timesteps])
    #chroma = np.reshape(chroma, [song_steps, chroma.shape[1]*num_timesteps])
       
    out_samples = sess.run(G_sample, feed_dict={Z: test_chroma})
    #print(np.shape(test_chroma),np.shape(samples))
    
    #print(np.floor(samples.shape[0]*samples.shape[1]/2/note_range).astype(int))
    
    S = np.reshape(out_samples, (np.floor(out_samples.shape[0]*out_samples.shape[1]/2/note_range).astype(int), 2*note_range))
    C = np.reshape(test_chroma, (test_chroma.shape[0]*num_timesteps, 12))
    #print(np.shape(S), np.shape(C))
    thresh_S = S>=0.5

    
    # plt.figure(figsize=(30,18))
    # plt.subplot(1,2,1)
    # plt.imshow(S)
    # plt.subplot(1,2,2)
    # plt.imshow(C)
    # #plt.tight_layout()
    # plt.pause(0.1)


    test = reverse_pianoroll.piano_roll_to_pretty_midi(convert.back(thresh_S), fs=16)
    test.write('converted/{}.mid'.format(i))

    # midi_manipulation.noteStateMatrixToMidi(thresh_S, "new/generated_chord_{}".format(i))
    # i+=1
    